In [ ]:
import tensorflow as tf
import pandas as pd

from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2

from tensortrade.strategies import StableBaselinesTradingStrategy
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
from tensortrade.actions import PairCriteriaSizeActions
from tensortrade.instruments import Quantity, TradingPair, BTC, USD
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.features.stationarity import FractionalDifference
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

WINDOW_SIZE = 20
PRICE_COLUMN = 'close'

normalize = MinMaxNormalizer(inplace=True)
difference = FractionalDifference(difference_order=0.6, inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize, difference])

action_scheme = PairCriteriaSizeActions(USD/BTC)
reward_scheme = RiskAdjustedReturns(return_algorithm="sharpe")

csv_file = tf.keras.utils.get_file(
    'Coinbase_BTCUSD_1h.csv', 'https://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv')
ohlcv_data = pd.read_csv(csv_file, skiprows=1, index_col="Date")
ohlcv_data.columns = map(str.lower, ohlcv_data.columns)
ohlcv_data = ohlcv_data.rename(columns={'volume btc': 'volume'})

# ohlcv_data = pd.read_csv('./data/updateOHLCdata_master.csv')
ohlcv_data = ohlcv_data.iloc[0:10000]

exchange = SimulatedExchange(data_frame=ohlcv_data, price_column=PRICE_COLUMN)

wallets = [(exchange, USD, 10000), (exchange, BTC, 0)]

portfolio = Portfolio(base_instrument=USD, wallet_tuples=wallets)

print('Transformed Data:')
print(exchange.observation_columns)

model = PPO2
policy = MlpLnLstmPolicy
params = {'learning_rate': 1e-5,
          'nminibatches': 1,
          'verbose': 2}

environment = TradingEnvironment(exchange=exchange,
                                 portfolio=portfolio,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 feature_pipeline=feature_pipeline,
                                 window_size=WINDOW_SIZE)

strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

performance = strategy.run(steps=10000)

W1208 17:55:37.715303 4464397760 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/module.py:31: The name tf.enable_resource_variables is deprecated. Please use tf.compat.v1.enable_resource_variables instead.

W1208 17:55:37.731585 4464397760 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:46: The name tf.train.AdadeltaOptimizer is deprecated. Please use tf.compat.v1.train.AdadeltaOptimizer instead.

W1208 17:55:37.732325 4464397760 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:47: The name tf.train.AdagradOptimizer is deprecated. Please use tf.compat.v1.train.AdagradOptimizer instead.

W1208 17:55:37.733123 4464397760 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tensorforce/core/optimizers/tf_optimizer.py:48: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train

Transformed Data:
Index(['open', 'high', 'low', 'close', 'volume', 'volume usd'], dtype='object')
Action: TradeSide.SELL | USD/BTC | 0.1111111111111111 | None
Balance Wallet: Balance: 0.00000000 BTC | Locked: 0.00000000 BTC
Order:  None
Portfolio:     BTC  BTC_locked      USD  USD_locked  net_worth step
0  0.0         0.0  10000.0         0.0    10000.0    0
Action: TradeSide.BUY | USD/BTC | 0.25 | None
Balance Wallet: Balance: 10000.00 USD | Locked: 0.00 USD
Order:  OrderStatus.PENDING | TradeSide.BUY | TradeType.MARKET | USD/BTC | 2500.00 USD | None | None
Before base wallet: Balance: 7500.00 USD | Locked: 2500.00 USD
Before quote wallet: Balance: 0.00000000 BTC | Locked: 0.00000000 BTC
After base wallet: Balance: 7500.00 USD | Locked: 7.48 USD
After quote wallet: Balance: 0.00000000 BTC | Locked: 0.21575115 BTC
Portfolio:          BTC  BTC_locked       USD  USD_locked     net_worth step
0  0.000000         0.0  10000.00         0.0  10000.000000    0
0  0.215751         0.0   7507.4

0  0.550541         0.0  3542.99         0.0  10021.504181   15
Action: TradeSide.SELL | USD/BTC | 0.5 | None
Balance Wallet: Balance: 0.55054108 BTC | Locked: 0.00000000 BTC
Order:  OrderStatus.PENDING | TradeSide.SELL | TradeType.MARKET | USD/BTC | 0.27527054 BTC | None | None
Before base wallet: Balance: 3542.99 USD | Locked: 0.00 USD
Before quote wallet: Balance: 0.27527054 BTC | Locked: 0.27527054 BTC
After base wallet: Balance: 3542.99 USD | Locked: 3220.84 USD
After quote wallet: Balance: 0.27527054 BTC | Locked: 0.00000000 BTC
Portfolio:          BTC  BTC_locked      USD  USD_locked     net_worth step
0  0.524877         0.0  3836.22         0.0   9992.576080   12
0  0.349918         0.0  5894.19         0.0  10022.519223   13
0  0.517299         0.0  3935.34         0.0   9971.182632   14
0  0.550541         0.0  3542.99         0.0  10021.504181   15
0  0.275271         0.0  6763.83         0.0   9994.366520   16
Action: TradeSide.SELL | USD/BTC | 0.1 | None
Balance Wallet: B

Action: TradeSide.BUY | USD/BTC | 0.2 | None
Balance Wallet: Balance: 5411.62 USD | Locked: 0.00 USD
Order:  OrderStatus.PENDING | TradeSide.BUY | TradeType.MARKET | USD/BTC | 1082.32 USD | None | None
Before base wallet: Balance: 4329.30 USD | Locked: 1082.32 USD
Before quote wallet: Balance: 0.38424942 BTC | Locked: 0.00000000 BTC
After base wallet: Balance: 4329.30 USD | Locked: 3.24 USD
After quote wallet: Balance: 0.38424942 BTC | Locked: 0.09101791 BTC
Portfolio:          BTC  BTC_locked      USD  USD_locked    net_worth step
0  0.717238         0.0  1513.62         0.0  9976.323029   28
0  0.759934         0.0  1010.59         0.0  9937.049515   29
0  0.768499         0.0   909.83         0.0  9923.599004   30
0  0.384249         0.0  5411.62         0.0  9926.954147   31
0  0.475267         0.0  4332.54         0.0  9950.323410   32
Action: TradeSide.SELL | USD/BTC | 0.3333333333333333 | None
Balance Wallet: Balance: 0.47526733 BTC | Locked: 0.00000000 BTC
Order:  OrderStatus.P

Action: TradeSide.SELL | USD/BTC | 0.16666666666666666 | None
Balance Wallet: Balance: 0.38517228 BTC | Locked: 0.00000000 BTC
Order:  OrderStatus.PENDING | TradeSide.SELL | TradeType.MARKET | USD/BTC | 0.06419538 BTC | None | None
Before base wallet: Balance: 5244.29 USD | Locked: 0.00 USD
Before quote wallet: Balance: 0.32097690 BTC | Locked: 0.06419538 BTC
After base wallet: Balance: 5244.29 USD | Locked: 742.70 USD
After quote wallet: Balance: 0.32097690 BTC | Locked: 0.00000000 BTC
Portfolio:          BTC  BTC_locked      USD  USD_locked    net_worth step
0  0.701930         0.0  1590.62         0.0  9751.667276   44
0  0.770345         0.0   797.69         0.0  9699.368046   45
0  0.577758         0.0  3020.97         0.0  9710.886743   46
0  0.385172         0.0  5244.29         0.0  9704.311530   47
0  0.320977         0.0  5986.99         0.0  9711.647675   48
Action: TradeSide.SELL | USD/BTC | 0.14285714285714285 | None
Balance Wallet: Balance: 0.32097690 BTC | Locked: 0.0000

...

0  0.000000         0.0  1113.38         0.0  1113.380000  3348
Action: TradeSide.SELL | USD/BTC | 0.2 | None
Balance Wallet: Balance: 0.00000000 BTC | Locked: 0.00000000 BTC
Order:  None
Portfolio:          BTC  BTC_locked      USD  USD_locked    net_worth  step
0  0.125345         0.0   615.42         0.0  1116.321916  3345
0  0.062672         0.0   865.45         0.0  1116.231262  3346
0  0.170199         0.0   434.01         0.0  1114.868454  3347
0  0.000000         0.0  1113.38         0.0  1113.380000  3348
0  0.000000         0.0  1113.38         0.0  1113.380000  3349
Action: TradeSide.SELL | USD/BTC | 0.5 | None
Balance Wallet: Balance: 0.00000000 BTC | Locked: 0.00000000 BTC
Order:  None
Portfolio:          BTC  BTC_locked      USD  USD_locked    net_worth  step
0  0.062672         0.0   865.45         0.0  1116.231262  3346
0  0.170199         0.0   434.01         0.0  1114.868454  3347
0  0.000000         0.0  1113.38         0.0  1113.380000  3348
0  0.000000         0.0 

0  0.219557         0.0  234.26         0.0  1112.675784  3363
Order:  None
Portfolio:          BTC  BTC_locked     USD  USD_locked    net_worth  step
0  0.188966         0.0  356.49         0.0  1107.955190  3360
0  0.200110         0.0  312.06         0.0  1107.429849  3361
0  0.207925         0.0  280.94         0.0  1106.400900  3362
0  0.219557         0.0  234.26         0.0  1112.675784  3363
0  0.219557         0.0  234.26         0.0  1110.888587  3364
Order:  None
Portfolio:          BTC  BTC_locked     USD  USD_locked    net_worth  step
0  0.200110         0.0  312.06         0.0  1107.429849  3361
0  0.207925         0.0  280.94         0.0  1106.400900  3362
0  0.219557         0.0  234.26         0.0  1112.675784  3363
0  0.219557         0.0  234.26         0.0  1110.888587  3364
0  0.219557         0.0  234.26         0.0  1110.043292  3365
Action: TradeSide.SELL | USD/BTC | 0.16666666666666666 | None
Balance Wallet: Balance: 0.21955729 BTC | Locked: 0.00000000 BTC
Orde

Portfolio:          BTC  BTC_locked     USD  USD_locked    net_worth  step
0  0.239257         0.0  154.14         0.0  1103.390588  3375
0  0.243542         0.0  137.06         0.0  1104.943702  3376
0  0.260693         0.0   68.74         0.0  1104.152398  3377
0  0.264146         0.0   55.03         0.0  1100.692542  3378
0  0.265872         0.0   48.17         0.0  1101.186530  3379
Action: TradeSide.SELL | USD/BTC | 0.5 | None
Balance Wallet: Balance: 0.26587231 BTC | Locked: 0.00000000 BTC
Order:  OrderStatus.PENDING | TradeSide.SELL | TradeType.MARKET | USD/BTC | 0.13293616 BTC | None | None
Before base wallet: Balance: 48.17 USD | Locked: 0.00 USD
Before quote wallet: Balance: 0.13293615 BTC | Locked: 0.13293616 BTC
After base wallet: Balance: 48.17 USD | Locked: 524.94 USD
After quote wallet: Balance: 0.13293615 BTC | Locked: 0.00000000 BTC
Portfolio:          BTC  BTC_locked     USD  USD_locked    net_worth  step
0  0.243542         0.0  137.06         0.0  1104.943702  3376


Portfolio:          BTC  BTC_locked     USD  USD_locked    net_worth  step
0  0.052285         0.0  889.22         0.0  1096.320085  3391
0  0.080168         0.0  778.40         0.0  1096.066035  3392
0  0.066806         0.0  831.19         0.0  1095.917709  3393
0  0.050105         0.0  897.23         0.0  1095.945136  3394
0  0.050105         0.0  897.23         0.0  1095.952651  3395
Action: TradeSide.SELL | USD/BTC | 0.1 | None
Balance Wallet: Balance: 0.05010480 BTC | Locked: 0.00000000 BTC
Order:  OrderStatus.PENDING | TradeSide.SELL | TradeType.MARKET | USD/BTC | 0.00501048 BTC | None | None
Before base wallet: Balance: 897.23 USD | Locked: 0.00 USD
Before quote wallet: Balance: 0.04509432 BTC | Locked: 0.00501048 BTC
After base wallet: Balance: 897.23 USD | Locked: 19.79 USD
After quote wallet: Balance: 0.04509432 BTC | Locked: 0.00000000 BTC
Portfolio:          BTC  BTC_locked     USD  USD_locked    net_worth  step
0  0.080168         0.0  778.40         0.0  1096.066035  3392

Portfolio:          BTC  BTC_locked     USD  USD_locked    net_worth  step
0  0.093282         0.0  725.58         0.0  1096.887215  3406
0  0.129458         0.0  580.89         0.0  1097.106346  3407
0  0.103567         0.0  684.24         0.0  1098.884658  3408
0  0.093210         0.0  725.53         0.0  1098.229213  3409
0  0.129423         0.0  580.85         0.0  1096.364448  3410
Action: TradeSide.BUY | USD/BTC | 0.16666666666666666 | None
Balance Wallet: Balance: 580.85 USD | Locked: 0.00 USD
Order:  OrderStatus.PENDING | TradeSide.BUY | TradeType.MARKET | USD/BTC | 96.81 USD | None | None
Before base wallet: Balance: 484.04 USD | Locked: 96.81 USD
Before quote wallet: Balance: 0.12942316 BTC | Locked: 0.00000000 BTC
After base wallet: Balance: 484.04 USD | Locked: 0.29 USD
After quote wallet: Balance: 0.12942316 BTC | Locked: 0.02425802 BTC
Portfolio:          BTC  BTC_locked     USD  USD_locked    net_worth  step
0  0.129458         0.0  580.89         0.0  1097.106346  3407
